In [1]:
import dspy
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS as LangchainFAISS
from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
class CustomFAISSRetriever(dspy.Retrieve):
    def load_index(self, idx_path=None, model_name:str=None):
        try:
            index = faiss.read_index(idx_path)
            print(f"成功載入FAISS索引，包含 {index.ntotal} 個向量")
            return index
        except Exception as e:
            print(f"索引載入失敗: {str(e)}")
            return None

    def load_local_db(self, local_db_path=None, embeddings=None):
        try:
            db = LangchainFAISS.load_local(
                folder_path=local_db_path,
                embeddings=embeddings,
                allow_dangerous_deserialization=True
            )
            print(f"載入成功，共 {db.index.ntotal} 筆技術問答")
            return db
        except Exception as e:
            print(f"向量庫載入異常: {str(e)}")
            return None
            
    def __init__(self, faiss_index_path, vector_db_path, k=2):
        super().__init__()
        self.k = k
        # 使用同一個模型名稱
        self.model_name = "paraphrase-multilingual-MiniLM-L12-v2"
        
        # 初始化 embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.model_name
        )
        
        # 載入 FAISS 索引
        self.index = self.load_index(faiss_index_path)
        
        # 載入向量庫
        self.vector_db = self.load_local_db(vector_db_path, self.embeddings)
        
        # 使用相同的模型進行查詢編碼
        self.model = SentenceTransformer(self.model_name)

    def __call__(self, query):
        # 編碼查詢
        query_embedding = self.model.encode(
            query,
            convert_to_tensor=False,
            show_progress_bar=False  # 對單一查詢關閉進度條
        )
        query_embedding = query_embedding.reshape(-1,1).T
        # query_embedding = query_embedding.cpu().numpy()
        query_embedding = query_embedding.astype(np.float32)
        
        # 搜索向量庫
        """
        question = input("Enter your query:")
        searchDocs = db.similarity_search(question)
        print(searchDocs[0].page_content)
        """
        # docs = self.vector_db.similarity_search_with_score(query_embedding, k=self.k)
        # docs = self.vector_db.similarity_search(query_embedding, k=self.k)
        distances, pos = self.index.search(query_embedding, k=self.k)
        # print(distance)
        # print(pos)
        # return the pos for retrieving data from answers
        return pos, distances;

In [3]:
# load_dotenv();
# model_name = os.getenv("SENTENCE_MODEL");
# print(f"mode is {model_name}");
# faiss_index_path = os.getenv("INDEX_PATH")
# print(f"Iindex is {faiss_index_path}");
# faiss_db_path = os.getenv("FAISS_DB")
# print(f"DB is {faiss_db_path}")

In [4]:
retriever=CustomFAISSRetriever(faiss_index_path="./db/20250307v1/dqe_learn_vdb_index.faiss", vector_db_path="./db/20250307v1/dqe_learn_vdb_faiss/")
if retriever:
    print("faiss retriever is created successfully");

成功載入FAISS索引，包含 28 個向量
載入成功，共 28 筆技術問答
faiss retriever is created successfully


In [5]:
# def search_similar_questions_by_vdb(retriever, question):
#     _docs= retriever(question)
#     print(f"{_docs[0].page_content}")
    # return pos, distances

In [6]:
# search_similar_questions_by_vdb(retriever, "电池鼓包的问题")

In [11]:
def search_similar_questions_by_index(retriever, question):
    pos, distances = retriever(question)
    return pos, distances

In [20]:
df = pd.read_csv("../../../../doc/deq_learn_refine2_correct.csv",encoding='utf-8-sig');

In [24]:
def getSubMessages(df_row):
    return {
        "question": df_row['问题现象描述'],
        "module": df_row['模块'],
        "severity": df_row['严重度'],
        "cause": df_row['原因分析'],
        "improve": df_row['改善对策'],
        "experience": df_row['经验萃取'],
        "judge": df_row['评审后优化'],
        "score": df_row['评分']
    }

In [25]:
def do_query_by_index(query:str=None):
    _pos, _distances = search_similar_questions_by_index(retriever, query)
    print(f"問題: {query}")
    print("\nAI-Response:")
    max_pos = np.argmax(_pos[0])
    count = 0
    _submessages = getSubMessages(df.iloc[max_pos]);
    print(_submessages)
    # for i in _pos[0]:
    #     count+=1;
    #     if i >= 0:
    #         print(f"第{count}筆回答：\n")
    #         print(f"模块:\n{df['模块'][i]}")
    #         print(f"严重度:\n{df['严重度'][i]}")
    #         print(f"问题现象描述:\n{df['问题现象描述'][i]}")
    #         print(f"原因分析:\n{df['原因分析'][i]}")
    #         print(f"改善对策:\n{df['改善对策'][i]}")
    #         print(f"经验萃取:\n{df['经验萃取'][i]}")
    #         print(f"评审后优化:\n{df['评审后优化'][i]}")
    #         print(f"评分:\n{df['评分'][i]}")
    #         print("=============================================================================================================")
    #     else:
    #         print(f"第{count}筆回答：\n")
    #         print("未找到资料")

In [26]:
question = "电池鼓包的问题"
do_query_by_index(query=question)

問題: 电池鼓包的问题

AI-Response:
{'question': '2022年9月19日，收到传音售后团队反馈，X1项目售后累计有将近200多块SSD不良补售后备品，寄不良样品回来分析，原因是江波龙的SSD G700型号主控引脚锡裂导致SSD不读盘；\n\n影响：X1项目SSD售后不良最终被定义为批量质量事故，导致客户市场对产品口碑的不良影响，最终按照批量事故处理，执行20W人民币的罚款；', 'module': '物料', 'severity': 'A', 'cause': '江波龙的SSD G700型号主控引脚锡裂导致SSD不读盘；不良图片如下图：', 'improve': '临时对策：江波龙对售后备品G700主控和Flash进行点胶工艺，以提高产品的鲁棒性，降低BGA Crack的风险。\n\n长期对策：因X1项目和G700型号SSD均已EOL,所以针对传音项目目前量产项目及后续在研项目的对应SSD均要求点胶后出货；', 'experience': '\n1.SSD工艺上优化，主控抗弯折强度增加，主控四周增加点胶，以提高产品鲁棒性（Robust）；\n点检人：袁新芳，负责SSD器件\n点检时间：关键器件评审前点检；\n\n2.强度可靠性测试规范优化：弯折测试要求23N 提升到30N\n点检人：袁新芳，负责SSD器件\n点检时间：新项目首次送样，弯折可靠性测试；\n\n\n', 'judge': '1. SSD主控点胶工艺默认在传音项目上check，在关键器件评审前点检\n', 'score': 4.0}
